# Preprocessing data for the Data Challenge

In [4]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=e23d095184074a7fc2df5fcffcc4247c9b14007ec456d793d29b29f0e8bb12cf
  Stored in directory: /Users/solenedebuysere/Library/Caches/pip/wheels/6a/67/f8/9cf1a8ff87e0b37f738769df49cc142a655489a6d27b68089f
Successfully built langdetect


In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

import time
import re
from langdetect import detect

from collections import Counter

import gc

In [6]:
FILENAME = "../data/train_series.csv"
FILENAME_ECB = "../data/ecb_data.csv"
FILENAME_FED = "../data/fed_data.csv"

In [7]:
ecb = pd.read_csv(FILENAME_ECB, index_col=0)
fed = pd.read_csv(FILENAME_FED, index_col=0)

In [8]:
ecb.head()

,title,speaker,text
0,Comments by Yves Mersch at Financial Services ...,Yves Mersch,Comments by Yves Mersch at Financial Service...
1,Securing sustained economic growth in the euro...,Vítor Constâncio,Securing sustained economic growth in the eu...
2,The role of monetary policy in addressing the ...,Mario Draghi,The role of monetary policy in addressing th...
3,The pandemic emergency: the three challenges f...,Philip R. Lane,SPEECH The pandemic emergency: the three c...
4,Transmission channels of monetary policy in th...,Peter Praet,Transmission channels of monetary policy in ...


In [9]:
fed.head()

,title,speaker,text
0,The Importance of Economic Education and Finan...,Governor Frederic S. Mishkin,As ...
1,Financial Innovation and Consumer Protection,Chairman Ben S. Bernanke,"The concept of financial innovation, it seems..."
2,Implementing Basel II in the United States,Governor Randall S. Kroszner,Good afternoon. I would like to thank Standar...
3,An Assessment of the U.S. Economy,Vice Chair for Supervision Randal K. Quarles,Thank you for the opportunity to take part in...
4,Monetary Policy since the Onset of the Crisis,Chairman Ben S. Bernanke,When we convened in Jackson Hole in August 20...


##### SUMMARIZE

In [6]:
import gensim.downloader as api
from gensim.summarization import summarize

In [7]:
text = (
"")
summarize(text)

''

In [8]:
text = (
    "Thomas A. Anderson is a man living two lives. By day he is an "
    "average computer programmer and by night a hacker known as "
    "Neo. Neo has always questioned his reality, but the truth is "
    "far beyond his imagination. Neo finds himself targeted by the "
    "police when he is contacted by Morpheus, a legendary computer "
    "hacker branded a terrorist by the government. Morpheus awakens "
    "Neo to the real world, a ravaged wasteland where most of "
    "humanity have been captured by a race of machines that live "
    "off of the humans' body heat and electrochemical energy and "
    "who imprison their minds within an artificial reality known as "
    "the Matrix. As a rebel against the machines, Neo must return to "
    "the Matrix and confront the agents: super-powerful computer "
    "programs devoted to snuffing out Neo and the entire human "
    "rebellion. "
)
print(text)
print(len(text.split()))

Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix. As a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion. 
137


In [9]:
shortText = summarize(text, ratio=0.3)
print(len(shortText.split()))

46


In [10]:
print(summarize(text, split=True))

["Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix."]


In [11]:
print(summarize(text, ratio=0.5))

By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination.
Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix.
As a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion.


In [12]:
len(str.split(ecb["text"][0]))

912

In [13]:
print(ecb["text"].apply(lambda x: len(str.split(str(x)))).median())
print(fed["text"].apply(lambda x: len(str.split(str(x)))).median())

2583.0
3444.0


In [14]:
print(ecb["text"][ecb["text"].isna()].index.tolist())
print(fed["text"][fed["text"].isna()].index.tolist())

[40, 104, 146, 172, 220, 257, 275, 291, 332, 413, 497, 509, 528, 649, 695, 739, 747, 753, 788, 829, 832, 1025, 1058, 1280, 1383, 1412, 1467, 1493, 1594, 1595, 1618, 1697, 1763, 1765]
[]


##### More complex text summarization

In [15]:
text = "My Wonderful Family\
I live in a house near the mountains. I have two brothers and one sister, and I was born last. My father teaches mathematics, and my mother is a nurse at a big hospital. My brothers are very smart and work hard in school. My sister is a nervous girl, but she is very kind. My grandmother also lives with us. She came from Italy when I was two years old. She has grown old, but she is still very strong. She cooks the best food!\
\
My family is very important to me. We do lots of things together. My brothers and I like to go on long walks in the mountains. My sister likes to cook with my grandmother. On the weekends we all play board games together. We laugh and always have a good time. I love my family very much."
print(text)

My Wonderful FamilyI live in a house near the mountains. I have two brothers and one sister, and I was born last. My father teaches mathematics, and my mother is a nurse at a big hospital. My brothers are very smart and work hard in school. My sister is a nervous girl, but she is very kind. My grandmother also lives with us. She came from Italy when I was two years old. She has grown old, but she is still very strong. She cooks the best food!My family is very important to me. We do lots of things together. My brothers and I like to go on long walks in the mountains. My sister likes to cook with my grandmother. On the weekends we all play board games together. We laugh and always have a good time. I love my family very much.


In [16]:
text = (
    "Thomas A. Anderson is a man living two lives. By day he is an "
    "average computer programmer and by night a hacker known as "
    "Neo. Neo has always questioned his reality, but the truth is "
    "far beyond his imagination. Neo finds himself targeted by the "
    "police when he is contacted by Morpheus, a legendary computer "
    "hacker branded a terrorist by the government. Morpheus awakens "
    "Neo to the real world, a ravaged wasteland where most of "
    "humanity have been captured by a race of machines that live "
    "off of the humans' body heat and electrochemical energy and "
    "who imprison their minds within an artificial reality known as "
    "the Matrix. As a rebel against the machines, Neo must return to "
    "the Matrix and confront the agents: super-powerful computer "
    "programs devoted to snuffing out Neo and the entire human "
    "rebellion. "
)
print(text)

Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix. As a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion. 


In [17]:
text = ecb["text"].iloc[0]
print(text)
print(len(text.split()))

  Comments by Yves Mersch at Financial Services summit   Comments by Yves Mersch, Member of the Executive Board of the ECB, Cumberland Lodge Financial Services Summit, 10 November 2016  Introduction   Sustainable economic growth in the real economy depends on the soundness of financial services and their providers;  This simple truth has been amply evidenced by the collapse of credit during the financial crisis and the subsequent recession. One important type of financial intermediary are central counterparties (CCPs);  Second, the financial ecosystem is – like the rest of the world - experiencing rapid change due to technical innovation;  I would like to give a clear signal that innovation should be embraced as the improvement of financial services can boost the real economy; but I also would like to give a warning that a certain level of caution is needed when technological innovations are applied to financial markets in order to avoid any negative consequences.   (i) Central counter

In [18]:
textt = summarize(text)
print(textt)
print(len(textt.split()))

One important type of financial intermediary are central counterparties (CCPs);  Second, the financial ecosystem is – like the rest of the world - experiencing rapid change due to technical innovation;  I would like to give a clear signal that innovation should be embraced as the improvement of financial services can boost the real economy; but I also would like to give a warning that a certain level of caution is needed when technological innovations are applied to financial markets in order to avoid any negative consequences.
In 2012 global standard setting bodies (CPMI-IOSCO) adopted key risk mitigation requirements – the Principles for Financial Market Infrastructures – which raised the bar worldwide for CCP supervision.
We are ready to explore all options within our mandate to ensure we can continue to fulfil our responsibilities under the Treaty, i.e. promoting the smooth operation of payment systems and ensuring the effective transmission of monetary policy, for which the stabil

##### Creation of ECB and FED summarization csv column

## Treating NA

In [19]:
ecb.isna().sum()

title       0
speaker     1
text       34
dtype: int64

In [20]:
fed.isna().sum()

title      0
speaker    2
text       0
dtype: int64

In [21]:
test_text = ecb.iloc[5]["text"]
print(test_text[:500])
print("...")
print(test_text[-5000:])

  Domestic and cross-border spillovers  of unconventional monetary policies   Remarks by Benoît Cœuré, Member of the Executive Board of the ECB, at the at the SNB-IMF Conference "Monetary Policy Challenges in a Changing World",Zurich, 12 May 2015  ***   Summary   Discussion has recently emerged on the global financial market implications of diverging monetary policy cycles. Central banks in large advanced economies can free themselves from the global financial cycle and regain monetary independe
...
feguard financial stability. Macro-prudential instruments can be targeted more efficiently to those sectors and countries where systemic risks may be materialising [14]. Finally, we encourage national authorities to do whatever is in their power to place the euro area on a more dynamic growth path, thereby creating attractive investment projects that generate high, but fundamentally justified, returns. These are the conditions for unconventional monetary policies to bring economies back to 

In [7]:
def numbered_reference_removal(text):
    # refs are typically in the form [n] in the text.
    matches = re.findall(r'\[[0-9]+?\]', text)
    counter = Counter(matches)
    for match in matches:
        if counter[match] != 2:
            # print(f"Didn't find a reference twice in the text, but {counter[match]} times. Cannot remove.")
            # print(text[:2000], '\n ... \n',text[-2000:])
            # print()
            return text
    
    assert len(matches)%2 == 0
    if len(matches) == 0:
        return text
    N = len(matches)//2
    is_ordered = True
    n = 0
    for s in matches:
        is_ordered = (s == f'[{n%N+1}]')
        if not is_ordered : 
            # print(f"Not ordered, {str(matches)}")
            return text
        n += 1
    
    # Remove all references after [1]
    res = re.findall(
        r'^.*?\[1\].+?\[1\]',
        text
    )
    return res[0]

In [8]:
def reference_removal_en(text):
    res = re.sub(r'(?i)see also.*', '', text)
    res = re.sub(r'References.*', '', res)
    res = re.sub("\[[0-9]+\]", "", res)
    return res

In [9]:
test_nb = 10
counter = 0
for text_ in ecb["text"]:
    if isinstance(text_, str):
        text = numbered_reference_removal(text_)
        text2 = reference_removal_en(text)
        if text2 != text:
            print(text[:500])
            print("...")
            print(text[-3000:])
            print("-------------")
            print(text2[:500])
            print("...")
            print(text2[-6000:])
            print("====================")
            counter += 1
    if counter == 10:
        break

  The role of monetary policy in addressing the crisis in the euro area   Speech by Mario Draghi, President of the ECB, at the “Room for discussion” of the Study Association SEFA and the Faculty of Economics and Business, Amsterdam,15 April 2013  Introduction There was a time, not too long ago, when central banking was considered to be a rather boring and unexciting occupation. In the era of the “Great Moderation”, mostly seen as the period between the mid-1980s and the beginning of the global f
...
et up to complement the national shock absorption capacity of euro area countries. The creation of the EFSF and more recently the ESM has addressed this last shortcoming. And in our joint work with the Presidents of the European Council, the European Commission and the Eurogroup, we have set out a vision and a process for building a genuine Economic and Monetary Union. This is designed to cover the other gaps in the institutional architecture that I previously referred to.  The genuine Econ

Sometimes, the title appears in the beginning of an ECB text. We can remove it using regular expressions.

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
from transformers import pipeline
## realSummarizer, not a shortener like summarize from gensim
realSummarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", device=device)

In [26]:
realSummarizer.device

device(type='cuda', index=0)

In [25]:
realSummarizer(text)

[{'summary_text': "Thomas A. Anderson is an average computer programmer and a hacker known as Neo. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy. Neo must return to the Matrix and confront the agents dedicated to snuffing out the human rebellion."}]

In [18]:
# Before dealing with N/A because otherwise, this edit would just undo the other.
# We can use this to remove the "TRANSCRIPT" and "SPEECH" tags.
# We also notice that most of the time, the content is preceded by a date. It would be useful to make a date parser to remove the first chunk.

def first_date_extractor(text):
    if len(text) > 0:
        res = re.sub('^(.*?)[1-9][0-9]* (?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|'
                     + 'Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?) (19|20)[0-9][0-9]',
                     '', text)
        return res

def remove_title(x):
    if not x["text"] is None and isinstance(x["text"], str):
        res = re.sub(x["title"], '', x["text"]).strip()
        return res
    else:
        return x["title"]

def website_remover(text):
    # Remove websites
    regex = "((http|https)://)(www.)?" \
        + "[a-zA-Z0-9@:%._\\+~#?&//=]{2,256}\\.[a-z]" \
        + "{2,6}\\b([-a-zA-Z0-9@:%._\\+~#?&//=]*)"
    res = re.sub(regex, "", text).strip()
    return res

def tag_removal(text):
    # Remove tags
    res = re.sub('SPEECH', '', text)
    res = re.sub('TRANSCRIPT', '', res)
    res = re.sub("Introduction", "", res)
    res = re.sub("Summary", "", res)
    return res

def summarizeLine(text, tolist=False):
    # Set approx 800 words when using the shortened text (func summarize of gensim)
    # as 800 corresponds to approx 1000 tokens which is minimum for the real summarizing model (hugging face bert)
    # This process come after a some preprocessing, so there are no NaN
    try:
        length_text = len(text.split())
        if length_text <= 800:
            return text
        
        factorOfSplit = 800/length_text
        res = summarize(text, ratio=factorOfSplit)
        res = ' '.join(res.split()[:850]) #trunc if the last sentence is really too big
        res = realSummarizer(' '.join(res.split()), max_length=511, min_length=510)
        res = res[0]["summary_text"]
        if tolist:
            res = res.split(".") #as the bart summarization separates sentences using dots (.)
    except: #if one sentence in text, summarize does not work as it selects only n most important sentences
        return text
    return res

def pipeline_en(x, tolist=False):
    res = remove_title(x)
    if res is None:
        return res
    res = numbered_reference_removal(res)
    res = reference_removal_en(res)
    res = tag_removal(res).strip()
    res = first_date_extractor(res).strip()
    # res = summarizeLine(res, tolist)

    # print("================================================")
    # print(res[:200])
    # print("...")
    # print(res[-500:])
    return res

ecb["text_preprocessed"] = ecb.apply(lambda x: pipeline_en(x, tolist=False), axis=1)

In [19]:
ecb["text_preprocessed"].iloc[:3].apply(lambda x: len(x.split())).describe()

count       3.000000
mean     2072.333333
std      1122.806454
min       883.000000
25%      1551.500000
50%      2220.000000
75%      2667.000000
max      3114.000000
Name: text_preprocessed, dtype: float64

In [27]:
ecb["text_preprocessed"].iloc[:3] = ecb["text_preprocessed"].iloc[:3].apply(lambda x: summarizeLine(x))

In [47]:
ecb["text_preprocessed"][ecb["text_preprocessed"].str.contains("Presentation")]

369     Les décisions de la BCE depuis la crise : un t...
600     Presentation of the ECB Annual Report 2019 to ...
687     Presentation slides It is a great pleasure to ...
700     This is the full text of a speech delivered in...
1398    Presentation slides I open this conference on ...
Name: text_preprocessed, dtype: object

In [30]:
realSummarizer(list(ecb["text_preprocessed"].iloc[4:7]))

Token indices sequence length is longer than the specified maximum sequence length for this model (3774 > 1024). Running this sequence through the model will result in indexing errors


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [119]:
len(ecb["text_preprocessed"].iloc[2].split())

410

In [16]:
ecb["text_preprocessed"].iloc[:3].apply(lambda x: len(x.split())).describe()

count       3.000000
mean     2072.333333
std      1122.806454
min       883.000000
25%      1551.500000
50%      2220.000000
75%      2667.000000
max      3114.000000
Name: text_preprocessed, dtype: float64

In [137]:
ecb["text_preprocessed"].iloc[0]

"The ECB is working as a catalyst for market integration. Central counterparties (CCPs) are systemically relevant. They have been in the focus of regulators and central banks since the financial crisis. The Eurosystem is responsible for operating TARGET2, the real-time gross settlement system for the euro, and T Target2-Securities, a service for the integrated settlement in central bank money of securities transactions. The smooth operation of these services is crucial for maintaining confidence in the euro and to support monetary policy operations. The ECB will keep a watchful eye on technical innovations and assess their possible impact on financial markets. The first condition is that innovations are proven to be safe before they find their way into the market. Second, it is equally important to ensure that innovations increase efficiency instead of a hurdle to increase efficiency and efficiency. ECB stands ready to work together with all interested parties in its catalyst towards t

Testing summarization function

In [20]:
text = (
    "Thomas A. Anderson is a man living two lives. By day he is an "
    "average computer programmer and by night a hacker known as "
    "Neo. Neo has always questioned his reality, but the truth is "
    "far beyond his imagination. Neo finds himself targeted by the "
    "police when he is contacted by Morpheus, a legendary computer "
    "hacker branded a terrorist by the government. Morpheus awakens "
    "Neo to the real world, a ravaged wasteland where most of "
    "humanity have been captured by a race of machines that live "
    "off of the humans' body heat and electrochemical energy and "
    "who imprison their minds within an artificial reality known as "
    "the Matrix. As a rebel against the machines, Neo must return to "
    "the Matrix and confront the agents: super-powerful computer "
    "programs devoted to snuffing out Neo and the entire human "
    "rebellion. "
)
print(text)

Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix. As a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion. 


In [13]:
ecb["text_preprocessed"].iloc[0]

'Sustainable economic growth in the real economy depends on the soundness of financial services and their providers;  This simple truth has been amply evidenced by the collapse of credit during the financial crisis and the subsequent recession. One important type of financial intermediary are central counterparties (CCPs);  Second, the financial ecosystem is – like the rest of the world - experiencing rapid change due to technical innovation;  I would like to give a clear signal that innovation should be embraced as the improvement of financial services can boost the real economy; but I also would like to give a warning that a certain level of caution is needed when technological innovations are applied to financial markets in order to avoid any negative consequences.   (i) Central counterparties   CCPs are systemically relevant. They have been in the focus of regulators and central banks since the financial crisis.  The growth in central clearing makes CCPs nodes for risk in the syste

In [71]:
summarize(text, 0.5)

"By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination.\nMorpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix.\nAs a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion."

In [19]:
np.argmax(ecb["text_preprocessed"].apply(lambda x: len(x.split())))

337

In [65]:
np.argmin(ecb["text_preprocessed"].apply(lambda x: len(x.split())))

80

In [67]:
print(ecb["text_preprocessed"].iloc[80])

Slides from the presentation


In [66]:
print(len(ecb["text_preprocessed"].iloc[0]))
print(len(ecb["text_preprocessed"].iloc[80]))
print(len(ecb["text_preprocessed"].iloc[337]))

5673
28
78174


In [52]:
print(len(ecb["text_preprocessed"].iloc[0].split()))
print(len(ecb["text_preprocessed"].iloc[337].split()))
ratioo = (800)/(len(ecb["text_preprocessed"].iloc[0].split()))
print(ratioo)
print(len( summarize(ecb["text_preprocessed"].iloc[0], ratio=ratioo).split() ))

883
13232
0.9060022650056625
803


In [57]:
' '.join(ecb["text_preprocessed"].iloc[337].split())

'Jump to the transcript of the questions and answers Mr Chairman, Honourable Members of the ECON committee, Ladies and Gentlemen, I am happy to be back to this committee for my first regular hearing at the European Parliament in the year 2015. A lot has happened since we last met in November last year. With Lithuania, the euro welcomed its 19th member. On 22 January we announced our intention to extend our asset purchase programmes to buying public sector securities; we started the purchases on March 9. We also moved to a new building which we officially inaugurated last week; we unveiled our new 20 euro banknote; and in a milestone towards even greater transparency of our decision-making procedures, we published on 19 February for the first time the accounts of a monetary policy meeting of the Governing Council of the ECB. I know this has always been a topic very dear to your committee. In the remainder of my remarks, I will explain some important aspects of the extended asset purchas

In [12]:
test_text = ecb.iloc[5]["text"]
print(test_text[:500])
print("...")
print(test_text[-500:])

  Domestic and cross-border spillovers  of unconventional monetary policies   Remarks by Benoît Cœuré, Member of the Executive Board of the ECB, at the at the SNB-IMF Conference "Monetary Policy Challenges in a Changing World",Zurich, 12 May 2015  ***   Summary   Discussion has recently emerged on the global financial market implications of diverging monetary policy cycles. Central banks in large advanced economies can free themselves from the global financial cycle and regain monetary independe
...
efficiency imply, from a long-run perspective, that the real growth rate exceeds the real interest rate on physical capital.    [13] Tirole, J. ( 1985): “Asset Bubbles and Overlapping Generations” Econometrica, Vol. 53, No. 6 (Nov., 1985), pp. 1499-1528    [14]“Reinforcing financial stability in the euro area”, speech given by Vítor Constâncio, Vice-President of the ECB, at the OMFIF City Lecture, London, 8 May 2015.       SEE ALSO  Find out more about related content   Related information 

In [13]:
print(textSummarized[0]["summary_text"].split("."))

['The ECB is working as a catalyst for market integration', ' Central counterparties (CCPs) are systemically relevant', ' They have been in the focus of regulators and central banks since the financial crisis', ' The Eurosystem is responsible for operating TARGET2, the real-time gross settlement system for the euro, and T Target2-Securities, a service for the integrated settlement in central bank money of securities transactions', ' The smooth operation of these services is crucial for maintaining confidence in the euro and to support monetary policy operations', ' The ECB will keep a watchful eye on technical innovations and assess their possible impact on financial markets', ' The first condition is that innovations are proven to be safe before they find their way into the market', ' Second, it is equally important to ensure that innovations increase efficiency instead of a hurdle to increase efficiency and efficiency', ' ECB stands ready to work together with all interested parties 

In [14]:
textSummarized = realSummarizer(' '.join(ecb["text_preprocessed"].iloc[337].split()[:850]), max_length=700, min_length=600) #does not work without join the split

In [15]:
test_nb = 10
counter = 0
for text_ in ecb["text"]:
    if isinstance(text_, str):
        text = numbered_reference_removal(text_)
        text2 = reference_removal_en(text)
        if text2 != text:
            print(text[:500])
            print("...")
            print(text[-300:])
            print("-------------")
            print(text2[:500])
            print("...")
            print(text2[-600:])
            print("====================")
            counter += 1
    if counter == 10:
        break

  The role of monetary policy in addressing the crisis in the euro area   Speech by Mario Draghi, President of the ECB, at the “Room for discussion” of the Study Association SEFA and the Faculty of Economics and Business, Amsterdam,15 April 2013  Introduction There was a time, not too long ago, when central banking was considered to be a rather boring and unexciting occupation. In the era of the “Great Moderation”, mostly seen as the period between the mid-1980s and the beginning of the global f
...
dy taken lies ahead. All parties involved in this comprehensive reform agenda must persevere. And we must all work with conviction and determination towards our shared vision. If we do so, I am confident that restoring stability and ensuring prosperity in the euro area are within our reach.      [1]
-------------
  The role of monetary policy in addressing the crisis in the euro area   Speech by Mario Draghi, President of the ECB, at the “Room for discussion” of the Study Association SEFA a

In [15]:
ecb["text_preprocessed"].iloc[0][0]["summary_text"]

'The ECB will keep a watchful eye on technical innovations and assess their possible impact on financial markets. Central counterparties (CCPs) have been in the focus of regulators and central banks since the financial crisis. The Eurosystem is looking for ways to improve the efficiency and lower the costs of its market infrastructure. ECB uses its position as broker to facilitate the harmonisation of market standards.'

In [16]:
ecb["text_preprocessed"].iloc[0]

[{'summary_text': 'The ECB will keep a watchful eye on technical innovations and assess their possible impact on financial markets. Central counterparties (CCPs) have been in the focus of regulators and central banks since the financial crisis. The Eurosystem is looking for ways to improve the efficiency and lower the costs of its market infrastructure. ECB uses its position as broker to facilitate the harmonisation of market standards.'}]

What are the N/A entries for text? Do they also have no speaker or no title?

In [17]:
ecb[ecb.isna().any(axis=1)]

,title,speaker,text,text_preprocessed
40,"Die EZB, das Geld und die Wirtschaft: Von A wi...",Sabine Lautenschläger,NaN,"Die EZB, das Geld und die Wirtschaft: Von A wi..."
104,Economic situation and outlook,Peter Praet,NaN,Economic situation and outlook
146,Economic developments in the euro area,Peter Praet,NaN,Economic developments in the euro area
172,Unequal scars – distributional consequences of...,Isabel Schnabel,NaN,Unequal scars – distributional consequences of...
220,The economic outlook for the euro area,Philip R. Lane,NaN,The economic outlook for the euro area
257,Mehr Europa für eine stabile gemeinsame Währung,Isabel Schnabel,NaN,Mehr Europa für eine stabile gemeinsame Währung
275,Better Regulation“ im Finanzsektor - die Sicht...,Gertrude Tumpel-Gugerell,NaN,Better Regulation“ im Finanzsektor - die Sicht...
291,The ECB's monetary policy strategy review - IM...,Philip R. Lane,NaN,The ECB's monetary policy strategy review - IM...
332,Sources of risk and vulnerabilities for financ...,Luis de Guindos,NaN,Sources of risk and vulnerabilities for financ...
413,Policy Frameworks and Strategies for an Open E...,Philip R. Lane,NaN,Policy Frameworks and Strategies for an Open E...


In [18]:
fed[fed.isna().any(axis=1)]

,title,speaker,text
461,Reflections on a Year of Crisis,NaN,Chairman Bernanke delivered the same remarks ...
494,Liquidity Provision by the Federal Reserve,NaN,Chairman Bernanke presented identical remarks...


FED is okay. In ECB, if there is no text, there is at least a title, so we can still work with that.

Let us view the speakers.

In [19]:
ecb["speaker"].value_counts()

Jean-Claude Trichet            216
Benoît Cœuré                   191
Mario Draghi                   186
Yves Mersch                    161
Peter Praet                    129
Vítor Constâncio               126
Lorenzo Bini Smaghi            105
Gertrude Tumpel-Gugerell        99
Sabine Lautenschläger           85
José Manuel González-Páramo     84
Jürgen Stark                    80
Luis de Guindos                 57
Jörg Asmussen                   51
Lucas Papademos                 45
Isabel Schnabel                 42
Christine Lagarde               41
Philip R. Lane                  40
Fabio Panetta                   26
Frank Elderson                   7
Name: speaker, dtype: int64

In [20]:
fed["speaker"].value_counts()

Chairman Ben S. Bernanke                        145
Governor Lael Brainard                           68
Governor Daniel K. Tarullo                       54
Governor Jerome H. Powell                        50
Vice Chairman Stanley Fischer                    45
Governor Elizabeth A. Duke                       44
Chair Janet L. Yellen                            44
Vice Chairman Donald L. Kohn                     41
Vice Chair for Supervision Randal K. Quarles     39
Governor Randall S. Kroszner                     36
Chairman Jerome H. Powell                        32
Governor Frederic S. Mishkin                     26
Vice Chairman Richard H. Clarida                 26
Vice Chair Janet L. Yellen                       22
Governor Sarah Bloom Raskin                      17
Governor Kevin Warsh                             16
Governor Jeremy C. Stein                         16
Governor Michelle W. Bowman                      13
Governor Susan S. Bies                            3
Name: speake

Let us view the text languages.

In [21]:
ecb["lang"] = ecb["text_preprocessed"].apply(lambda x : detect(x[:500]), )
fed["lang"] = fed["text"].apply(lambda x : detect(x[:500]))

In [22]:
ecb["lang"].value_counts()

en    1646
de      75
fr      31
es      16
it       4
Name: lang, dtype: int64

In [23]:
fed["lang"].value_counts()

en    739
Name: lang, dtype: int64

In [24]:
ecb[ecb["lang"] != "en"]["text_preprocessed"].str.len().sum(skipna=True)

2565448

In [25]:
ecb.loc[31]["text"]

'  Die Finanzmarktunion als Element einer stabilen Währungsunion?   Rede von Jörg Asmussen, Mitglied des Direktoriums der EZB, Handelsblatt Jahrestagung „Banken im Umbruch“, Frankfurt am Main, 4. September 2012 Sehr geehrte Damen und Herren,  Ein Satz in der Schlusserklärung des Euro-Gipfels vom 28. Juni dieses Jahres sorgte für Aufhorchen und viel Diskussionsstoff über den Sommer – ich zitiere: „ Sobald unter Einbeziehung der EZB ein wirksamer einheitlicher Aufsichts\xadmechanismus für Banken des Euro-Währungs\xadgebiets eingerichtet worden ist, hätte der ESM nach einem ordentlichen Beschluss die Möglich\xadkeit, Banken direkt zu rekapitalisieren.“  Hier wurde der Anfang einer Finanzmarktunion beschlossen. Als Beitrag zu dieser Diskussion möchte ich im Folgenden gerne drei Aspekte aufgreifen:    Wo stehen wir bei der Finanzmarkt\xadregulierung? Wie stellt sich aktuell die Lage an den Finanzmärkten dar?   Warum ist eine Finanzmarktunion not\xadwendig geworden? Wie soll diese aus\xadseh